In [1]:
import sys
import gymnasium
import os
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor

from building_energy_storage_simulation import BuildingSimulation, Environment

In [2]:
def read_data():
    load = pd.read_csv('../building_energy_storage_simulation/data/preprocessed/electricity_load_profile.csv')[
        'Load [kWh]']
    price = pd.read_csv('../building_energy_storage_simulation/data/preprocessed/electricity_price_profile.csv')[
        'Day Ahead Auction']
    generation = pd.read_csv('../building_energy_storage_simulation/data/preprocessed/solar_generation_profile.csv')[
        'Generation [kWh]']
    return np.array(load), np.array(price), np.array(generation)

In [3]:
class ObservationWrapper(gymnasium.Wrapper):
    def __init__(self, env, forecast_length):
        super().__init__(env)
        
        self.forecast_length = forecast_length
        original_observation_space_length = self.observation_space.shape[0]
        self.observation_space = gymnasium.spaces.Box(shape=(original_observation_space_length - forecast_length,),
                                                low=-np.inf,
                                                high=np.inf, dtype=np.float32)

    def reset(self, seed: int = 42, options=None):
        obs, info = self.env.reset()
        return self.convert_observation(obs), info

    def step(self, action):

        obs, reward, done, trunc, info = self.env.step(action)
        return self.convert_observation(obs), reward, done, trunc, info

    def convert_observation(self, obs):
        load_forecast = obs[1: self.forecast_length + 1]
        generation_forecast = obs[self.forecast_length + 1: 2 * self.forecast_length + 1]
        price_forecast = obs[2 * self.forecast_length + 1: 3 * self.forecast_length + 1]
        soc = obs[0]
        return np.concatenate(([soc],
                               load_forecast - generation_forecast,
                               price_forecast),
                              axis=0)
        

# Applying Reiforcement Learning Using Stable Baselines 3


In [4]:
RL_PATH = 'rl_example/'
os.makedirs(RL_PATH, exist_ok=True)

# Create Environment
sim = BuildingSimulation(max_battery_charge_per_timestep=50, battery_capacity=200)
env = Environment(sim, num_forecasting_steps=8, max_timesteps=4380)
env = ObservationWrapper(env, 8)
initial_obs, info = env.reset()
print(initial_obs)

[ 0.       9.89     9.08     8.22     8.57     8.93     9.2     10.71
  3.98309  5.005    4.133    4.322    4.546    3.767    3.97     4.059
  4.326  ]


In [5]:
# Wrap with Monitor() so a log of the training is saved 
env = Monitor(env, filename=RL_PATH)
# Warp with DummyVecEnc() so the observations and reward can be normalized using VecNormalize()
env = DummyVecEnv([lambda: env])
env = VecNormalize(env, norm_obs=True, norm_reward=True)

In [ ]:
# Train with PPO :-)
model = SAC("MlpPolicy", env, verbose=1, gamma=0.95)
model.learn(total_timesteps=200000)
# Store the trained Model and environment stats (which are needed as we are standardizing the observations and reward using VecNormalize())
model.save(RL_PATH + 'model')
env.save(RL_PATH + 'env.pkl')

Using cpu device


In [ ]:
env.save(RL_PATH + 'env.pkl')

# Evaluation

In [ ]:
# Plot the training process
training_log = pd.read_csv(RL_PATH + 'monitor.csv', skiprows=1)
training_log['r'].plot()

In [ ]:
load, price, generation = read_data()
load = load[4380:6080]
price = price[4380:6080]
generation = generation[4380:6080]

max_timesteps = 6000 - 4380

eval_sim = BuildingSimulation(electricity_load_profile=load,
                              solar_generation_profile=generation,
                              electricity_price=price,
                              max_battery_charge_per_timestep=50, 
                              battery_capacity=200)
eval_env = Environment(eval_sim, num_forecasting_steps=8, max_timesteps=max_timesteps)
eval_env = ObservationWrapper(eval_env, 8)
eval_env = DummyVecEnv([lambda: eval_env])
# It is important to load the environmental statistics here as we use a rolling mean calculation !
eval_env = VecNormalize.load(RL_PATH + 'env.pkl', eval_env)     

In [ ]:
eval_env.training = False

actions, observations, electricity_consumption, excess_energy, rewards = ([], [], [], [], [])
done = False
obs = eval_env.reset()
while not done:
        action = model.predict(obs, deterministic=True)
        obs, r, done, info = eval_env.step([action[0][0]])

        actions.append(action[0][0][0])
        original_reward = eval_env.get_original_reward()[0]
        original_obs = eval_env.get_original_obs()[0]
        observations.append(original_obs)
        electricity_consumption.append(info[0]['electricity_consumption'])
        excess_energy.append(info[0]['electricity_price'])
        rewards.append(r)
        
trajectory = pd.DataFrame({
        'action': actions,
        'observations': observations,
        'electricity_consumption': electricity_consumption,
        'electricity_price': excess_energy,
        'reward': rewards
    })        

In [ ]:
plot_data = trajectory[0:200]
observation_df = plot_data['observations'].apply(pd.Series)

plt.rcParams["figure.figsize"] = (16,10)

fig, ax = plt.subplots()
ax.plot(observation_df[1], label = 'Residual Load')
ax.plot(plot_data['electricity_price'], label = 'Electricity Price')

ax1 = ax.twinx()
ax1.plot(plot_data['action'], label = 'action', color = 'black')
fig.legend(bbox_to_anchor=[0.5, 0.95], loc = 'center', ncol=5, prop={'size': 16})

# Compare to Baseline

In [ ]:
eval_env.training = False

cost = []
done = False
obs = eval_env.reset()
while not done:
        action = model.predict(obs, deterministic=True)
        obs, r, done, info = eval_env.step([action[0][0]])
        cost.append(info[0]['electricity_consumption'] * info[0]['electricity_price'])

cost = sum(cost)

In [ ]:
eval_env.training = False

baseline_cost = []
done = False
obs = eval_env.reset()
while not done:
        # Always taking noop as action. This is the electricity demand if there would be no battery
        action = [0]
        obs, r, done, info = eval_env.step(action)
        baseline_cost.append(info[0]['electricity_consumption'] * info[0]['electricity_price'])

baseline_cost = sum(baseline_cost)

In [ ]:
# how much energy did we save by utilizing the battery?
1 -(cost / baseline_cost)

In [ ]:
baseline_cost

In [ ]:
1 - 0.8971223120327491